# ACRA results analysis

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, FormatStrFormatter

In this document, we generate the relevant metrics for each experiment. In particular, we use as relevant metrics the accuracy, the average utility gain, FPR and FNR.

##  Auxiliar functions

We define here the relevant metrics to be calculated for each experiment.

In [2]:
def postolabel(posterior, ut):
    aux = np.dot(ut, posterior.transpose())   
    return(np.argmax(aux, axis = 0))

In [3]:
def avgUt(y_true, y_pred, ut):
    y_true = y_true.astype(int)
    y_pred = y_pred.astype(int)
    return( float(sum(ut[y_pred,y_true])) / float(len(y_true) ))

In [4]:
def acc(y_true, y_pred):
    return(accuracy_score(y_true, y_pred))

In [5]:
def FPR(y_true, y_pred):
    a = confusion_matrix(y_true, y_pred)
    return(float(a[0,1]) / float(sum(a[0,:])))

In [6]:
def FNR(y_true, y_pred):
    a = confusion_matrix(y_true, y_pred)
    return(float(a[1,0])/ float(sum((a[1,:]))))

## Results dataframe generation

For a given `N` and give `var`, this function reads the corresponding experiment, computes all relevant metrics, and returns a dataframe with the results.

In [7]:
def getMetrics(N, var):


    ut01 = [0, -1, -5, -10]
    columns = ["N", "var", "Ut01", "NBC_Acc", "ACRA_Acc", "NB_Acc", "NBC_AvgUt", "ACRA_AvgUt", "NB_AvgUt",\
               "NBC_FPR", "ACRA_FPR", "NB_FPR", "NBC_FNR", "ACRA_FNR", "NB_FNR"]
    resultsdf = pd.DataFrame(data=None, columns=columns)

    name = "IgnAtt" + "N" + str(N) + "var" + str(var) + ".csv"
    dataPath = "results/results_uci_1gwi_severalAtt/"
    data = pd.read_csv(dataPath + name)

    for i in range(len(ut01)):

        if ut01[i] == 0:
            ut = np.array([[1,0],[0,1]])
            NB_label = postolabel(data[["NBpost0", "NBpost1"]].values, ut)
            NBC_label = postolabel(data[["NBCpost0", "NBCpost1"]].values, ut)
            ACRA_label = postolabel(data[["ACRApost0", "ACRApost1"]].values, ut)

            resultsdf.loc[i] = [N, var, ut01[i], acc(data.spam.values, NBC_label), acc(data.spam.values, ACRA_label), \
                          acc(data.spam.values, NB_label), avgUt(data.spam.values, NBC_label, ut), \
                          avgUt(data.spam.values, ACRA_label, ut), avgUt(data.spam.values, NB_label, ut), \
                          FPR(data.spam.values, NBC_label), FPR(data.spam.values, ACRA_label),\
                          FPR(data.spam.values, NB_label), FNR(data.spam.values, NBC_label), \
                          FNR(data.spam.values, ACRA_label), FNR(data.spam.values, NB_label)]
        else:

            ut = np.array([[1,-1],[ut01[i],1]])
            NB_label = postolabel(data[["NBpost0", "NBpost1"]].values, ut)
            NBC_label = postolabel(data[["NBCpost0", "NBCpost1"]].values, ut)
            ACRA_label = postolabel(data[["ACRApost0", "ACRApost1"]].values, ut)

            resultsdf.loc[i] = [N, var, ut01[i], acc(data.spam.values, NBC_label), acc(data.spam.values, ACRA_label), \
                          acc(data.spam.values, NB_label), avgUt(data.spam.values, NBC_label, ut), \
                          avgUt(data.spam.values, ACRA_label, ut), avgUt(data.spam.values, NB_label, ut), \
                          FPR(data.spam.values, NBC_label), FPR(data.spam.values, ACRA_label),\
                          FPR(data.spam.values, NB_label), FNR(data.spam.values, NBC_label), \
                          FNR(data.spam.values, ACRA_label), FNR(data.spam.values, NB_label)]


    return(resultsdf)



In [8]:
getMetrics(0,10.0)

,N,var,Ut01,NBC_Acc,ACRA_Acc,NB_Acc,NBC_AvgUt,ACRA_AvgUt,NB_AvgUt,NBC_FPR,ACRA_FPR,NB_FPR,NBC_FNR,ACRA_FNR,NB_FNR
0,0.0,10.0,0.0,0.891399,0.857515,0.881842,0.891399,0.857515,0.881842,0.070588,0.057353,0.070588,0.163482,0.265393,0.186837
1,0.0,10.0,-1.0,0.891399,0.857515,0.881842,0.782798,0.715030,0.763684,0.070588,0.057353,0.070588,0.163482,0.265393,0.186837
2,0.0,10.0,-5.0,0.894874,0.857515,0.883579,0.671590,0.614248,0.649001,0.050000,0.042647,0.050000,0.184713,0.286624,0.212314
3,0.0,10.0,-10.0,0.888792,0.857515,0.879235,0.550825,0.543006,0.531712,0.042647,0.032353,0.042647,0.210191,0.301486,0.233546


## Compute metrics for all experiments

We compute the metrics for all experiments and write the results.

In [9]:
N = range(10)
var = [1.0,10.0,20.0,30.0,40.0,50.0,60.0,70.0,80.0,90.0]

result = getMetrics(0, 1.0)
for i in N:
    for j in var:
        if not (i == 0 and j == 1.0):
            tmp = getMetrics(i, j)
            result = result.append(tmp, ignore_index=True)

#result.to_csv("results/whole.csv")

## Take mean values

We compute the average value of the ten experiments done under different training-test splits.

In [98]:
means = result.groupby(['var', 'Ut01'], as_index=False)["NBC_Acc", "ACRA_Acc", "NB_Acc", "NBC_AvgUt", "ACRA_AvgUt", "NB_AvgUt",\
               "NBC_FPR", "ACRA_FPR", "NB_FPR", "NBC_FNR", "ACRA_FNR", "NB_FNR"].mean()
deviations = result.groupby(['var', 'Ut01'], as_index=False)["NBC_Acc", "ACRA_Acc", "NB_Acc", "NBC_AvgUt", "ACRA_AvgUt", "NB_AvgUt",\
               "NBC_FPR", "ACRA_FPR", "NB_FPR", "NBC_FNR", "ACRA_FNR", "NB_FNR"].std().fillna(result.groupby(['var', 'Ut01'], as_index=False).last())
deviations
#means.to_csv("results/means.csv")

/Users/eagle/anaconda3/lib/python3.6/site-packages/pandas/core/groupby.py:1083: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(self.var(ddof=ddof, **kwargs))


,var,Ut01,NBC_Acc,ACRA_Acc,NB_Acc,NBC_AvgUt,ACRA_AvgUt,NB_AvgUt,NBC_FPR,ACRA_FPR,NB_FPR,NBC_FNR,ACRA_FNR,NB_FNR
0,1.000000,-10.0,0.013843,0.074357,0.009872,0.048743,0.119506,0.040962,0.004774,0.007842,0.004774,0.028308,0.204673,0.019221
1,1.000000,-5.0,0.013004,0.073054,0.010538,0.035787,0.127632,0.029672,0.005296,0.010435,0.005296,0.026783,0.204744,0.021341
2,1.000000,-1.0,0.010955,0.073232,0.009751,0.021910,0.146465,0.019501,0.006803,0.012536,0.006803,0.020450,0.205905,0.021208
3,1.000000,0.0,0.010955,0.073232,0.009751,0.010955,0.073232,0.009751,0.006803,0.012536,0.006803,0.020450,0.205905,0.021208
4,3.162278,-10.0,0.013843,0.076690,0.009872,0.048743,0.130421,0.040962,0.004774,0.007592,0.004774,0.028308,0.209136,0.019221
5,3.162278,-5.0,0.013004,0.074983,0.010538,0.035787,0.135537,0.029672,0.005296,0.009798,0.005296,0.026783,0.207209,0.021341
6,3.162278,-1.0,0.010955,0.075103,0.009751,0.021910,0.150206,0.019501,0.006803,0.012920,0.006803,0.020450,0.209695,0.021208
7,3.162278,0.0,0.010955,0.075103,0.009751,0.010955,0.075103,0.009751,0.006803,0.012920,0.006803,0.020450,0.209695,0.021208
8,4.472136,-10.0,0.013843,0.074827,0.009872,0.048743,0.119382,0.040962,0.004774,0.008607,0.004774,0.028308,0.206448,0.019221
9,4.472136,-5.0,0.013004,0.073726,0.010538,0.035787,0.129546,0.029672,0.005296,0.010334,0.005296,0.026783,0.205950,0.021341


## Analysis Plots

In [12]:
def scatterPlot(x,y1,y2,y3,l, xl, yl, t,name):


    f,(ax,ax2) = plt.subplots(2,1,sharex=True, facecolor='w')
    
    
    # Autoscale y axis and set format en float
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax2.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    
    
    sns.set_style("whitegrid")
    # plot the same data on both axes
    ax.plot(x, y1, color = sns.xkcd_rgb["pale red"])
    ax.plot(x, y2, color = sns.xkcd_rgb["denim blue"])
    ax2.plot(x, y3, color = sns.xkcd_rgb["medium green"])

    ax.legend(labels=[l[0], l[1]])
    ax2.legend(handles=ax2.lines[::len(ut0["ACRA_Acc"])+1], labels=[l[2]])
    #ax.set_xticklabels([t.get_text().split("T")[0] for t in ax.get_xticklabels()])
    plt.gcf().autofmt_xdate()
    ax2.set(xlabel=xl)
    ax2.set_ylabel(yl)
    ax2.yaxis.set_label_coords(0.04, 0.47, transform=f.transFigure)

    f.suptitle(t) 

    # hide the spines between ax and ax2
    ax.spines['bottom'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax.xaxis.tick_top()
    ax.tick_params(labeltop='off')  # don't put tick labels at the top
    ax2.xaxis.tick_bottom()

    # This looks pretty good, and was fairly painless, but you can get that
    # cut-out diagonal lines look with just a bit more work. The important
    # thing to know here is that in axes coordinates, which are always
    # between 0-1, spine endpoints are at these locations (0,0), (0,1),
    # (1,0), and (1,1).  Thus, we just need to put the diagonals in the
    # appropriate corners of each of our axes, and so long as we use the
    # right transform and disable clipping.

    d = .015  # how big to make the diagonal lines in axes coordinates
    # arguments to pass to plot, just so we don't keep repeating them
    kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
    ax.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
    ax.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

    kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
    ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
    ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal

    # What's cool about this is that now if we vary the distance between
    # ax and ax2 via f.subplots_adjust(hspace=...) or plt.subplot_tool(),
    # the diagonal lines will move accordingly, and stay right at the tips
    # of the spines they are 'breaking'
    
    
    plt.savefig(name, format='eps', dpi=1000)

    return(plt)

In [13]:
def scatterPlot2(x,y1,y2,y3,l, xl, yl, t, name):


    f,(ax,ax2) = plt.subplots(2,1,sharex=True, facecolor='w')
    
    # Autoscale y axis and set format en float
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax2.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    
    
    sns.set_style("whitegrid")
    # plot the same data on both axes
    ax2.plot(x, y1, color = sns.xkcd_rgb["pale red"])
    ax2.plot(x, y2, color = sns.xkcd_rgb["denim blue"])
    ax.plot(x, y3, color = sns.xkcd_rgb["medium green"])

    ax2.legend(labels=[l[0], l[1]])
    ax.legend(labels=[l[2]])
    #ax.set_xticklabels([t.get_text().split("T")[0] for t in ax.get_xticklabels()])
    plt.gcf().autofmt_xdate()
    ax2.set(xlabel=xl)
    ax2.set_ylabel(yl)
    ax2.yaxis.set_label_coords(0.04, 0.47, transform=f.transFigure)

    f.suptitle(t) 


    # hide the spines between ax and ax2
    ax.spines['bottom'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax2.xaxis.tick_top()
    ax2.tick_params(labeltop='off')  # don't put tick labels at the top
    ax2.xaxis.tick_bottom()

    # This looks pretty good, and was fairly painless, but you can get that
    # cut-out diagonal lines look with just a bit more work. The important
    # thing to know here is that in axes coordinates, which are always
    # between 0-1, spine endpoints are at these locations (0,0), (0,1),
    # (1,0), and (1,1).  Thus, we just need to put the diagonals in the
    # appropriate corners of each of our axes, and so long as we use the
    # right transform and disable clipping.

    d = .015  # how big to make the diagonal lines in axes coordinates
    # arguments to pass to plot, just so we don't keep repeating them
    kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
    ax.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
    ax.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

    kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
    ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
    ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal

    # What's cool about this is that now if we vary the distance between
    # ax and ax2 via f.subplots_adjust(hspace=...) or plt.subplot_tool(),
    # the diagonal lines will move accordingly, and stay right at the tips
    # of the spines they are 'breaking'
    plt.savefig(name, format='eps', dpi=1000)

    return(plt)

In [14]:
def scatterPlot3(x,y1,y2,y3,l, xl, yl, t, name):


    f,ax2 = plt.subplots(1,1,sharex=True, facecolor='w')
    sns.set_style("whitegrid")
    # plot the same data on both axes
    ax2.plot(x, y1, color = sns.xkcd_rgb["pale red"])
    ax2.plot(x, y3, color = sns.xkcd_rgb["medium green"])
    ax2.plot(x, y2, color = sns.xkcd_rgb["denim blue"])
    

    ax2.legend(labels=[l[0], l[1],l[2]])
    plt.gcf().autofmt_xdate()
    ax2.set(xlabel=xl)
    ax2.set_ylabel(yl)
    #ax.yaxis.set_label_coords(0.04, 0.47, transform=fig.transFigure)

    f.suptitle(t)
    
    plt.savefig(name, format='eps', dpi=1000)




    return(plt)

### Plot accuracies for different utilities

In [131]:
def scatterPlotTesting(x,y1,y2,y3,y1varHigh, y1varLow, l, xl, yl, t,name):
    f,(ax,ax2) = plt.subplots(2,1,sharex=True, facecolor='w')
    # Autoscale y axis and set format en float
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax2.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    
    sns.set_style("whitegrid")
    
    # plot the same data on both axes
    ax.plot(x, y1, color = sns.xkcd_rgb["pale red"])
    ax.fill_between(x,y1varLow,y1varHigh,facecolor='LightBlue',alpha=0.5)
    
    ax.plot(x, y2, color = sns.xkcd_rgb["denim blue"])
    ax2.plot(x, y3, color = sns.xkcd_rgb["medium green"])

    ax.legend(labels=[l[0], l[1]])
    ax2.legend(handles=ax2.lines[::len(ut0["ACRA_Acc"])+1], labels=[l[2]])
    #ax.set_xticklabels([t.get_text().split("T")[0] for t in ax.get_xticklabels()])
    plt.gcf().autofmt_xdate()
    
    ax2.set(xlabel=xl)
    ax2.set_ylabel(yl)
    ax2.yaxis.set_label_coords(0.04, 0.47, transform=f.transFigure)

    f.suptitle(t) 

    # hide the spines between ax and ax2
    ax.spines['bottom'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax.xaxis.tick_top()
    ax.tick_params(labeltop='off')  # don't put tick labels at the top
    ax2.xaxis.tick_bottom()

    d = .015  # how big to make the diagonal lines in axes coordinates
    # arguments to pass to plot, just so we don't keep repeating them
    kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
    ax.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
    ax.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal
    
    

    kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
    ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
    ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal
    
    plt.savefig(name, format='eps', dpi=1000)

    return(plt)

In [133]:
def scatterPlotTesting2(x,y1,y2,y3,y1varHigh, y1varLow,l, xl, yl, t, name):
    f,(ax,ax2) = plt.subplots(2,1,sharex=True, facecolor='w')
    
    # Autoscale y axis and set format en float
    ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    ax2.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    
    sns.set_style("whitegrid")
    # plot the same data on both axes
    ax2.plot(x, y1, color = sns.xkcd_rgb["pale red"])
    ax2.fill_between(x,y1varLow,y1varHigh,facecolor='LightBlue',alpha=0.5)
    
    ax2.plot(x, y2, color = sns.xkcd_rgb["denim blue"])
    ax.plot(x, y3, color = sns.xkcd_rgb["medium green"])

    ax2.legend(labels=[l[0], l[1]])
    ax.legend(labels=[l[2]])
   
    plt.gcf().autofmt_xdate()
    ax2.set(xlabel=xl)
    ax2.set_ylabel(yl)
    ax2.yaxis.set_label_coords(0.04, 0.47, transform=f.transFigure)

    f.suptitle(t) 


    # hide the spines between ax and ax2
    ax.spines['bottom'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax2.xaxis.tick_top()
    ax2.tick_params(labeltop='off')  # don't put tick labels at the top
    ax2.xaxis.tick_bottom()

    d = .015  # how big to make the diagonal lines in axes coordinates
    # arguments to pass to plot, just so we don't keep repeating them
    kwargs = dict(transform=ax.transAxes, color='k', clip_on=False)
    ax.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
    ax.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

    kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
    ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
    ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal

    plt.savefig(name, format='eps', dpi=1000)

    return(plt)

In [134]:
def scatterPlotTesting3(x,y1,y2,y3,y1varHigh, y1varLow,l, xl, yl, t, name):

    f,ax2 = plt.subplots(1,1,sharex=True, facecolor='w')    
    sns.set_style("whitegrid")
    # plot the same data on both axes
    ax2.plot(x, y1, color = sns.xkcd_rgb["pale red"])
    ax2.fill_between(x,y1varLow,y1varHigh,facecolor='LightBlue',alpha=0.5)
    
    ax2.plot(x, y3, color = sns.xkcd_rgb["medium green"])
    ax2.plot(x, y2, color = sns.xkcd_rgb["denim blue"])
    

    ax2.legend(labels=[l[0], l[1],l[2]])
    plt.gcf().autofmt_xdate()
    ax2.set(xlabel=xl)
    ax2.set_ylabel(yl)
    #ax.yaxis.set_label_coords(0.04, 0.47, transform=fig.transFigure)

    f.suptitle(t)
    
    plt.savefig(name, format='eps', dpi=1000)

    return(plt)

In [84]:
#ut0 = means[means.Ut01 == 0.0]
#ut0["var"]
#std0 = deviations[deviations.Ut01 == 0.0]
#std0["NBC_Acc"]

3     0.010955
7     0.010955
11    0.010955
15    0.010955
19    0.010955
23    0.010955
27    0.010955
31    0.010955
35    0.010955
39    0.010955
Name: NBC_Acc, dtype: float64

In [85]:
ut0 = means[means.Ut01 == 0.0]
ut0["var"]

p = scatterPlotTesting(ut0["var"],ut0["ACRA_Acc"],ut0["NBC_Acc"],ut0["NB_Acc"],ut0["ACRA_Acc"]+std0["ACRA_Acc"],ut0["ACRA_Acc"]-std0["ACRA_Acc"],["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Accuracy",\
               "0-1 Utility", "plots/testing/test1.eps")

p.show()


/Users/eagle/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [93]:
means
#std1 = deviations[deviations.Ut01 == -1.0]
#std1["ACRA_Acc"]

,var,Ut01,NBC_Acc,ACRA_Acc,NB_Acc,NBC_AvgUt,ACRA_AvgUt,NB_AvgUt,NBC_FPR,ACRA_FPR,NB_FPR,NBC_FNR,ACRA_FNR,NB_FNR
0,1.0,-10.0,0.885230,0.819461,0.875152,0.517116,0.473936,0.496959,0.046359,0.030224,0.046359,0.220296,0.414569,0.246181
1,1.0,-5.0,0.889140,0.821112,0.878367,0.652129,0.553606,0.630582,0.051942,0.036517,0.051942,0.201845,0.400730,0.229298
2,1.0,-1.0,0.890443,0.820417,0.881668,0.780886,0.640834,0.763336,0.067971,0.050959,0.067971,0.173676,0.380179,0.196096
3,1.0,0.0,0.890443,0.820417,0.881668,0.890443,0.820417,0.881668,0.067971,0.050959,0.067971,0.173676,0.380179,0.196096
4,10.0,-10.0,0.885230,0.818940,0.875152,0.517116,0.465856,0.496959,0.046359,0.031508,0.046359,0.220296,0.413958,0.246181
5,10.0,-5.0,0.889140,0.821025,0.878367,0.652129,0.552737,0.630582,0.051942,0.036812,0.051942,0.201845,0.400504,0.229298
6,10.0,-1.0,0.890443,0.820591,0.881668,0.780886,0.641182,0.763336,0.067971,0.052382,0.067971,0.173676,0.377542,0.196096
7,10.0,0.0,0.890443,0.820591,0.881668,0.890443,0.820591,0.881668,0.067971,0.052382,0.067971,0.173676,0.377542,0.196096
8,20.0,-10.0,0.885230,0.821112,0.875152,0.517116,0.474109,0.496959,0.046359,0.030797,0.046359,0.220296,0.409438,0.246181
9,20.0,-5.0,0.889140,0.823110,0.878367,0.652129,0.557950,0.630582,0.051942,0.036380,0.051942,0.201845,0.395741,0.229298


In [135]:
ut0 = means[means.Ut01 == 0.0]
std0 = deviations[deviations.Ut01 == 0.0]
#ut0["var"]

#p = scatterPlot(ut0["var"],ut0["ACRA_Acc"],ut0["NBC_Acc"],ut0["NB_Acc"],\
#                ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Accuracy",\
#               "0-1 Utility", "plots/sevAtt/ign_accUt0.eps")

p = scatterPlotTesting(ut0["var"],ut0["ACRA_Acc"],ut0["NBC_Acc"],ut0["NB_Acc"],ut0["ACRA_Acc"]+std0["ACRA_Acc"],ut0["ACRA_Acc"]-std0["ACRA_Acc"],["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Accuracy",\
               "0-1 Utility", "plots/sevAtt/ign_accUt0.eps")

p.show()

#########################################################################################
#########################################################################################


ut1 = means[means.Ut01 == -1.0]
std1 = deviations[deviations.Ut01 == -1.0]
#q = scatterPlot(ut1["var"],ut1["ACRA_Acc"],ut1["NBC_Acc"],ut1["NB_Acc"],\
#                ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Accuracy",\
#               "U(+,-) = -1","plots/sevAtt/ign_accUt1.eps")

q = scatterPlotTesting(ut1["var"],ut1["ACRA_Acc"],ut1["NBC_Acc"],ut1["NB_Acc"],ut1["ACRA_Acc"]+std1["ACRA_Acc"],ut1["ACRA_Acc"]-std1["ACRA_Acc"],["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Accuracy","U(+,-) = -1","plots/sevAtt/ign_accUt1.eps")


q.show()
#########################################################################################
#########################################################################################

ut5 = means[means.Ut01 == -5.0]
std5 = deviations[deviations.Ut01 == -5.0]
#r = scatterPlot(ut5["var"],ut5["ACRA_Acc"],ut5["NBC_Acc"],ut5["NB_Acc"],\
#                ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Accuracy",\
#               "U(+,-) = -5", "plots/sevAtt/ign_accUt5.eps")

r = scatterPlotTesting(ut5["var"],ut5["ACRA_Acc"],ut5["NBC_Acc"],ut5["NB_Acc"],ut5["ACRA_Acc"]+std5["ACRA_Acc"],ut5["ACRA_Acc"]-std5["ACRA_Acc"],\
                ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Accuracy",\
               "U(+,-) = -5", "plots/sevAtt/ign_accUt5.eps")

r.show()
#########################################################################################
#########################################################################################


ut10 = means[means.Ut01 == -10.0]
std10 = deviations[deviations.Ut01 == -10.0]
s = scatterPlotTesting(ut10["var"],ut10["ACRA_Acc"],ut10["NBC_Acc"],ut10["NB_Acc"],ut10["ACRA_Acc"]+std10["ACRA_Acc"],ut10["ACRA_Acc"]-std10["ACRA_Acc"],\
                ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Accuracy",\
               "U(+,-) = -10", "plots/sevAtt/ign_accUt10.eps")

s.show()

/Users/eagle/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [104]:
ut10["ACRA_Acc"]+std10["ACRA_Acc"]

0     0.893819
4     0.895630
8     0.895939
12    0.895397
16    0.896605
20    0.895809
24    0.897527
28    0.897288
32    0.897133
36    0.896939
Name: ACRA_Acc, dtype: float64

### Plot average utility 

In [136]:
#p = scatterPlot(ut0["var"],ut0["ACRA_AvgUt"],ut0["NBC_AvgUt"],ut0["NB_AvgUt"],\
#                ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Average Utility",\
#               "0-1 Utility", "plots/sevAtt/ign_avut0.eps")

p = scatterPlotTesting(ut0["var"],ut0["ACRA_AvgUt"],ut0["NBC_AvgUt"],ut0["NB_AvgUt"],ut0["ACRA_AvgUt"]+std0["ACRA_AvgUt"],ut0["ACRA_AvgUt"]-std0["ACRA_AvgUt"],\
                ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Average Utility",\
               "0-1 Utility", "plots/sevAtt/ign_avut0.eps")

p.show()

#########################################################################################
#########################################################################################


q = scatterPlotTesting(ut1["var"],ut1["ACRA_AvgUt"],ut1["NBC_AvgUt"],ut1["NB_AvgUt"],ut1["ACRA_AvgUt"]+std1["ACRA_AvgUt"],ut1["ACRA_AvgUt"]-std1["ACRA_AvgUt"],\
                ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Average Utility",\
               "U(+,-) = -1", "plots/sevAtt/ign_avut1.eps")

q.show()
#########################################################################################
#########################################################################################


r = scatterPlotTesting(ut5["var"],ut5["ACRA_AvgUt"],ut5["NBC_AvgUt"],ut5["NB_AvgUt"],ut5["ACRA_AvgUt"]+std5["ACRA_AvgUt"],ut5["ACRA_AvgUt"]-std5["ACRA_AvgUt"],\
                ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Average Utility",\
               "U(+,-) = -5", "plots/sevAtt/ign_avut5.eps")

r.show()
#########################################################################################
#########################################################################################

s = scatterPlotTesting(ut10["var"],ut10["ACRA_AvgUt"],ut10["NBC_AvgUt"],ut10["NB_AvgUt"],ut10["ACRA_AvgUt"]+std10["ACRA_AvgUt"],ut10["ACRA_AvgUt"]-std10["ACRA_AvgUt"],\
                ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "Average Utility",\
               "U(+,-) = -10", "plots/sevAtt/ign_avut10.eps")

s.show()

/Users/eagle/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


### Plot FNR for different Utilities

In [137]:
#p = scatterPlot2(ut0["var"],ut0["ACRA_FNR"],ut0["NBC_FNR"],ut0["NB_FNR"],\
#                 ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "FNR",\
#               "0-1 Utility", "plots/sevAtt/ign_fnut0.eps")

p = scatterPlotTesting2(ut0["var"],ut0["ACRA_FNR"],ut0["NBC_FNR"],ut0["NB_FNR"],ut0["ACRA_FNR"]+std0["ACRA_FNR"],ut0["ACRA_FNR"]-std0["ACRA_FNR"],\
                 ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "FNR",\
               "0-1 Utility", "plots/sevAtt/ign_fnut0.eps")

p.show()

#########################################################################################
#########################################################################################


q = scatterPlotTesting2(ut1["var"],ut1["ACRA_FNR"],ut1["NBC_FNR"],ut1["NB_FNR"],ut1["ACRA_FNR"]+std1["ACRA_FNR"],ut1["ACRA_FNR"]-std1["ACRA_FNR"],\
                 ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "FNR",\
               "U(+,-) = -1", "plots/sevAtt/ign_fnut1.eps")

q.show()
#########################################################################################
#########################################################################################


r = scatterPlotTesting2(ut5["var"],ut5["ACRA_FNR"],ut5["NBC_FNR"],ut5["NB_FNR"],ut5["ACRA_FNR"]+std5["ACRA_FNR"],ut5["ACRA_FNR"]-std5["ACRA_FNR"],\
                 ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "FNR",\
               "U(+,-) = -5", "plots/sevAtt/ign_fnut5.eps")

r.show()
#########################################################################################
#########################################################################################

s = scatterPlotTesting2(ut10["var"],ut10["ACRA_FNR"],ut10["NBC_FNR"],ut10["NB_FNR"],ut10["ACRA_FNR"]+std10["ACRA_FNR"],ut10["ACRA_FNR"]-std10["ACRA_FNR"],\
                 ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "FNR",\
               "U(+,-) = -10", "plots/sevAtt/ign_fnut10.eps")

s.show()

/Users/eagle/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


### Plot FPR for different Utilities

In [138]:
#p = scatterPlot3(ut0["var"],ut0["ACRA_FPR"],ut0["NBC_FPR"],ut0["NB_FPR"],\
#             ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "FPR","0-1 Utility", \
#                 "plots/sevAtt/ign_fput0.eps")

p = scatterPlotTesting3(ut0["var"],ut0["ACRA_FPR"],ut0["NBC_FPR"],ut0["NB_FPR"],ut0["ACRA_FPR"]+std0["ACRA_FPR"],ut0["ACRA_FPR"]-std0["ACRA_FPR"],["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "FPR","0-1 Utility", "plots/sevAtt/ign_fput0.eps")

p.show()

#########################################################################################
#########################################################################################


q = scatterPlotTesting3(ut1["var"],ut1["ACRA_FPR"],ut1["NBC_FPR"],ut1["NB_FPR"],ut1["ACRA_FPR"]+std1["ACRA_FPR"],ut1["ACRA_FPR"]-std1["ACRA_FPR"],\
            ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "FPR","U(+,-) = -1", \
                 "plots/sevAtt/ign_fput1.eps")

q.show()
#########################################################################################
#########################################################################################


r = scatterPlotTesting3(ut5["var"],ut5["ACRA_FPR"],ut5["NBC_FPR"],ut5["NB_FPR"],ut5["ACRA_FPR"]+std5["ACRA_FPR"],ut5["ACRA_FPR"]-std5["ACRA_FPR"], \
            ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "FPR","U(+,-) = -5", \
                 "plots/sevAtt/ign_fput5.eps")

r.show()
#########################################################################################
#########################################################################################

s = scatterPlotTesting3(ut10["var"],ut10["ACRA_FPR"],ut10["NBC_FPR"],ut10["NB_FPR"],ut10["ACRA_FPR"]+std10["ACRA_FPR"],ut10["ACRA_FPR"]-std10["ACRA_FPR"], \
            ["ACRA", "NB-Plain", "NB-Tainted"], "k(%)", "FPR","U(+,-) = -10", \
                 "plots/sevAtt/ign_fput10.eps")

s.show()


/Users/eagle/anaconda3/lib/python3.6/site-packages/matplotlib/pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
